<a href="https://colab.research.google.com/github/fjavier141/proyecto-final-magister-ds/blob/main/exploracion_inicial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

import pandas as pd
import numpy as np
from dotenv import load_dotenv
from pathlib import Path
from sqlalchemy import create_engine, text

ROOT = Path().resolve().parent
load_dotenv(dotenv_path=ROOT / ".env")

# Parámetros de conexión (ajusta a tu entorno)
USER = os.getenv("POSTGRES_USER")
PASSWORD = os.getenv("POSTGRES_PASSWORD")
HOST = os.getenv("POSTGRES_HOST")
PORT = os.getenv("POSTGRES_PORT", "5432")
DB = os.getenv("POSTGRES_DB")

# Crea el motor SQLAlchemy
engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}")

# Test de conexión
try:
    with engine.connect() as conn:
        version = conn.execute(text("SELECT version();"))
        print("✅ Conectado a:", list(version)[0][0])
except Exception as e:
    print("❌ Error al conectar:", e)


✅ Conectado a: PostgreSQL 15.13 (Debian 15.13-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit


In [4]:
#Extracción data dimensional
with engine.connect() as conn:
    clientes = pd.read_sql(text("SELECT id_cliente, id_barrio, id_comuna, segmento, canal, descr_flag_patente FROM stg.base_clientes;"), conn)

with engine.connect() as conn:
    barrios = pd.read_sql(text("SELECT id_barrio, indice_gse, n_habitantes, n_ptos_interes, superficie_km2, densidad_hab FROM stg.info_distritos;"), conn)    

with engine.connect() as conn:
    macro_vars = pd.read_sql(text("SELECT id_periodo, uf, dolar, ipc, imacec, tpm, tasa_desempleo FROM stg.datos_macro;"), conn) 
    


In [5]:
#Extracción venta cervezas

with engine.connect() as conn:
    ventas_cervezas = pd.read_sql(text("SELECT id_categoria, id_cliente, id_periodo, tipo_mix, id_sku_venta, liq_um FROM stg.venta_historica WHERE id_categoria=1;"), conn)

In [8]:
# Preprocesamiento

print(ventas_cervezas.head())
print(ventas_cervezas.info())


   id_categoria  id_cliente  id_periodo tipo_mix  id_sku_venta   liq_um
0             1      719703      201811   MASIVO        450607  0.75600
1             1      719703      201811   MASIVO        450640  0.11760
2             1      719703      201811   MASIVO        450676  0.41748
3             1      719703      201811  PREMIUM          7485  0.11088
4             1      719703      201811  PREMIUM          7493  0.27720
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28327324 entries, 0 to 28327323
Data columns (total 6 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id_categoria  int64  
 1   id_cliente    int64  
 2   id_periodo    int64  
 3   tipo_mix      object 
 4   id_sku_venta  int64  
 5   liq_um        float64
dtypes: float64(1), int64(4), object(1)
memory usage: 1.3+ GB
None
